## A complete plotting example

This example is inspired from the [plot-gtk-ui](https://github.com/sumitsahrawat/plot-gtk-ui) package. Our goal will be to create an interface similar to the screenshot below.

![Screenshot](https://raw.githubusercontent.com/sumitsahrawat/plot-gtk-ui/master/screenshots/sinax.png)

In [1]:
{-# LANGUAGE OverloadedStrings #-}
import IHaskell.Display.Widgets

First, we create a common structure that will hold all the information required to create a plot. This has to be done first so that we can hook widget events to modify it. The plotting logic is implemented next for the same reason.

In [2]:
import Data.IORef
import Data.Monoid (mempty)
import Data.Text (Text)

data PlotInfo = PlotInfo {
    plotTitle :: Text,
    plotTitleSize :: Integer,
    subTitle :: Text,
    subTitleSize :: Integer,
    xLabel :: Text,
    xLabelSize :: Integer,
    yLabel :: Text,
    yLabelSize :: Integer,
    showXAxis :: Bool,
    showYAxis :: Bool,
    xRange :: (Double, Double),
    yRange :: (Double, Double)
  }

defaultPlotInfo = PlotInfo {
    plotTitle = mempty,
    plotTitleSize = 10,
    subTitle = mempty,
    subTitleSize = 10,
    xLabel = mempty,
    xLabelSize = 10,
    yLabel = mempty,
    yLabelSize = 10,
    showXAxis = True,
    showYAxis = True,
    xRange = (-5, 5),
    yRange = (-5, 5)
  }

plotData <- newIORef defaultPlotInfo

# TODO: Plotting Implementation

The first required element is a box, to implement a vertical division between the plotting region and the input widgets.

In [3]:
divBox <- mkFlexBox
setField divBox Orientation HorizontalOrientation

-- Two parts: A FlexBox for the left part (plot + sliders) and an Accordion for the input elements.
plBox <- mkFlexBox
tlBox <- mkAccordion

-- Add the widgets to the main dividing box.
setField divBox Children [ChildWidget plBox, ChildWidget tlBox]

-- Make the orientation Vertical
setField plBox Orientation VerticalOrientation

Now we fill in the plotting area with:

+ A `FlexBox` to hold the sliders.
+ An `ImageWidget` to hold the plot.

In [4]:
slBox <- mkFlexBox
plImg <- mkImageWidget

-- Sliders need to be laid out vertically.
setField slBox Orientation VerticalOrientation

-- Add widgets to the plotting region.
setField plBox Children [ChildWidget slBox, ChildWidget plImg]

Now, we fill the other half with the following:

+ Four `FlexBox` widgets (title, sub-title, x-label, y-label), containing a `TextWidget` for title and a `BoundedIntText` for the font size.
+ A `FlexBox` with two selection widgets for toggling axis visibility. We'll go with `ToggleButton` just for fun.
+ Two more `FlexBox`, with `FloatText` widgets for deciding the plot range.

In [5]:
-- The four FlexBox widgets.
import Control.Monad (replicateM, forM_)
import Data.List (zip4)
import Text.Printf (printf)
import Data.Text (pack)

-- pl : plotTitle
-- sb : subTitle
-- x : xLabel
-- y : yLabel
boxes <- replicateM 4 mkFlexBox
texts@[plTxt,sbTxt,xTxt,yTxt] <- replicateM 4 mkTextWidget
inpts@[plInp,sbInp,xInp,yInp] <- replicateM 4 mkBoundedIntText

-- Adding event handlers. This is a clumsy way to emulate first-class record fields.
let setHandler widget fieldSetter = setField widget SubmitHandler $ do
      oldVal <- readIORef plotData
      newStr <- getField widget StringValue
      writeIORef plotData (fieldSetter oldVal newStr)
 in do
   setHandler plTxt $ \struct val -> struct { plotTitle = val }
   setHandler sbTxt $ \struct val -> struct { subTitle = val }
   setHandler xTxt $ \struct val -> struct { xLabel = val }
   setHandler yTxt $ \struct val -> struct { yLabel = val }

let boxInfo = zip4 boxes texts inpts ["plot title", "sub-title", "X-Label", "Y-Label"]

forM_ boxInfo $ \(box,text,input,placeholder) -> do
  setField box Orientation HorizontalOrientation
  setField box Children [ChildWidget text, ChildWidget input]
  setField text Placeholder $ pack $ printf "Enter %s here ..." placeholder
  setField input MinInt 1
  setField input MaxInt 72
  setField input IntValue 10

In [6]:
-- A FlexBox with ToggleButtons
buttonBox <- mkFlexBox
setField buttonBox Orientation HorizontalOrientation
tButtons@[xBut,yBut] <- replicateM 2 mkToggleButton

let tgButtonInfo = zip tButtons ["X-Axis", "Y-Axis"]

let setHandler widget fieldSetter = setField widget ChangeHandler $ do
      oldVal <- readIORef plotData
      newStr <- getField widget BoolValue
      writeIORef plotData (fieldSetter oldVal newStr)
 in do
   setHandler xBut $ \struct val -> struct { showXAxis = val }
   setHandler yBut $ \struct val -> struct { showYAxis = val }

forM_ tgButtonInfo $ \(widget, description) -> do
  setField widget Description description
  setField widget BoolValue True

setField buttonBox Children (map ChildWidget tButtons)

In [7]:
-- Finally, the ranges
rangeBoxes <- replicateM 2 mkFlexBox
fTxts@[xLow,xHigh,yLow,yHigh] <- replicateM 4 mkFloatText

let rangeInfo = zip rangeBoxes [(xLow,xHigh), (yLow, yHigh)]

forM_ rangeInfo $ \(box, (lowTxt, highTxt)) -> do
  setField box Orientation HorizontalOrientation
  setField box Children (map ChildWidget [lowTxt, highTxt])

Now, to finally add these widgets to the right part of the window.

In [8]:
setField tlBox Children $ map ChildWidget $ boxes ++ [buttonBox] ++ rangeBoxes

We also need to give a title to each page in the `Accordion` widget.

In [9]:
setField tlBox Titles ["Plot title", "Subtitle", "X-Label", "Y-Label", "Axis visibility", "X-range", "Y-range"]

Then we sync the values from out `plotData :: IORef PlotData` to the widgets.

In [10]:
let syncVal widget value fieldGetter = readIORef plotData >>= setField widget value . fieldGetter
 in do
   syncVal plTxt StringValue plotTitle
   syncVal plInp IntValue plotTitleSize
   syncVal sbTxt StringValue subTitle
   syncVal sbInp IntValue subTitleSize
   
   syncVal xTxt StringValue xLabel
   syncVal xInp IntValue xLabelSize
   syncVal yTxt StringValue yLabel
   syncVal yInp IntValue yLabelSize

   syncVal xBut BoolValue showXAxis
   syncVal yBut BoolValue showYAxis
   
   syncVal xLow FloatValue (fst . xRange)
   syncVal xHigh FloatValue (snd . xRange)
   syncVal yLow FloatValue (fst . yRange)
   syncVal yHigh FloatValue (snd . yRange)

In [11]:
divBox